# Settle Up Problem (Dlužníček)

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [1]:
P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values())/len(c)  # the settlement value

Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x_k, y_k, z_k)$ representing the transactions: person $x_k$ should pay person $y_k$ z_k euros. The number of transactions (i.e., the length of the list) should be minimized.

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [2]:
import gurobipy as g  # import Gurobi module


# model --------------------------------------------------
m = g.Model()

# - ADD VARIABLES
f = {}
y = {}
for p in P:
    for q in P:
        # f_pq represents how much money should p pay to q
        f[p, q] = m.addVar(lb=0, vtype=g.GRB.CONTINUOUS)
        # y_pq represents whether p should pay to q or not
        y[p, q] = m.addVar(vtype=g.GRB.BINARY)

# - ADD CONSTRAINTS
sv = sum(c.values()) / len(c)
for p in P:
    # what person gets, payed and pays sums up to settlement value
    m.addConstr(g.quicksum(f[p, q] for q in P) + c[p] - g.quicksum(f[q, p] for q in P) == sv)

M = sum(c.values())

for p in P:
    for q in P:
        # link variables y with f
        m.addConstr(f[p, q] <= y[p, q] * M)
        m.addConstr(y[p, q] <= f[p, q])

# - SET OBJECTIVE
# minimize the number of transactions
m.setObjective(g.quicksum(y[p, q] for p in P for q in P), sense=g.GRB.MINIMIZE)

# call the solver -------------------------------------------
m.optimize()

# print the solution -----------------------------------------
print('\nSOLUTION:')
for p in P:
    for q in P:
        if f[p, q].x > 0:
            print("{0} -> {1}: {2}".format(p, q, f[p, q].x))

#print([y[p, q].x for p in P for q in P])
# always round results before casting to int!
#print([int(round(f[p, q].x)) for p in P for q in P])

Academic license - for non-commercial use only
Optimize a model with 36 rows, 32 columns and 88 nonzeros
Variable types: 16 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolved: 28 rows, 24 columns, 72 nonzeros
Variable types: 12 continuous, 12 integer (12 binary)

Root relaxation: objective 3.900000e-01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.39000    0    3    3.00000    0.39000  87.0%     -    0s
     0     0    2.00000    0    3    3.00000    2.00000  33.3%     -    0s

Cutting planes:
  MIR: 1
  Flow cover: 7

Explored 1 nodes (17 simplex iterations) in 0.03 seconds
Thread 

## Additional experiments

* experiment with different values of the 'big M' constant
* try to generalize the model to work even with float sv (e.g., rounded to 0.01)

More comments can be found in `settle_up_models.ipynb.zip` from the Lab 3.
 